<a href="https://colab.research.google.com/github/MrJangoBox/Team-13-Padelol-Kaggle-IFT-6578/blob/main/Public_Score_1_69462_IFT_6578_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.linear_model import LinearRegression, Ridge, LassoLars
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, MaxAbsScaler, FunctionTransformer
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, GenericUnivariateSelect
from sklearn.feature_selection import SelectFpr, SelectFdr, SelectFwe, SelectPercentile
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import VotingRegressor

### Load data

In [ ]:
# Load data
if 'google.colab' in str(get_ipython()):
    files = []
    files.append(('test.csv',
                'https://drive.google.com/uc?id=1gu6DXcEr4agiGttFj2ubToSICl5NDGKw'))
    files.append(('train.csv',
                'https://drive.google.com/uc?id=1x-Baw1riR-7wKZTpAvXe9RCCC0zu1sXv'))

    for file in files:
        gdown.download(file[1], file[0], quiet=True)

    train_data, test_data = pd.read_csv('train.csv'), pd.read_csv('test.csv')
else:
    train_data, test_data = pd.read_csv('../train.csv'), pd.read_csv('../test.csv')

# Rename columns
train_cols = train_data.columns.str.lower().str.replace(' ', '_').str.replace('?', '')
test_cols = train_cols[:-1]
train_data.columns = train_cols
test_data.columns = test_cols
# Split train dataset to train and validation
X_train, X_valid, y_train, y_valid = train_test_split(train_data.iloc[:, :-1],
                                                      train_data.iloc[:, -1],
                                                      test_size=0.01,
                                                      shuffle=True,
                                                      random_state=76)

In [ ]:
X_train

,id,user_name,personal_url,profile_cover_image_status,profile_verification_status,profile_text_color,profile_page_color,profile_theme_color,is_profile_view_size_customized,utc_offset,location,location_public_visibility,user_language,profile_creation_timestamp,user_time_zone,num_of_followers,num_of_people_following,num_of_status_updates,num_of_direct_messages,profile_category,avg_daily_profile_visit_duration_in_seconds,avg_daily_profile_clicks,profile_image
4412,546I1ENF9Q41LRGR,kTbNbgc2crg,https://blob/u19rbujwg.com,Set,Pending,ebc712,777777,ffcc33,False,-14400.0,"UCF, Orlando, Florida",enabled,en,Mon May 11 15:00:37 +0000 2009,Eastern Time (US & Canada),3355,204,871,74,unknown,26.680,3.7832,546I1ENF9Q41LRGR.png
2179,N0N9G39RGSI2RPKM,aLiiQJvoO_Vzu,https://blob/ouqM5.f_pd.com,Not set,Verified,2fc2ef,252429,1a1b1f,False,-18000.0,Im on Location,Disabled,en,Wed Apr 22 04:10:38 +0000 2009,Central Time (US & Canada),888002,224,8406,990,government,39.597,1.7103,N0N9G39RGSI2RPKM.png
2771,DTGKI9967R8OAB6J,EfV1rOfgQ,https://blob/9txpmluFmT.com,Set,Verified,090066,ddeef6,000000,False,7200.0,Imprint:,Enabled,en,Fri Mar 19 14:30:32 +0000 2010,Bern,385523,32,81845,2156,celebrity,17.721,3.5157,DTGKI9967R8OAB6J.png
599,MS94MPB6E37QDJB5,i3hO1g4hSXe4EE,https://blob/rkl42chf/4.com,Not set,Verified,1f3a46,000000,000000,False,-25200.0,Ciudad de México,Disabled,es,Mon Jun 27 16:20:50 +0000 2016,Pacific Time (US & Canada),10811,121,5602,10,unknown,23.779,4.8063,MS94MPB6E37QDJB5.png
683,9DP8QPCLA90H0BIL,0C7g0yyhaxuF,NaN,Set,Verified,009999,efefef,131516,False,19800.0,NaN,enabled,en,Mon Aug 17 07:52:17 +0000 2009,Mumbai,960784,771,20275,3467,business,29.958,4.3430,9DP8QPCLA90H0BIL.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6426,O2F5PK0KCN65OT97,tO4VVjdcYbre,https://blob/Wv5e_k99mx.com,Set,Pending,0084b4,c0dfec,013861,False,-10800.0,Chile,enabled,es,Fri Sep 16 13:17:04 +0000 2011,Santiago,114845,844,65285,311,business,43.926,2.8904,O2F5PK0KCN65OT97.png
6528,C0QTG3P4A2DICQ7O,EMe3V5_nyAS7a,NaN,Set,Verified,234e99,c7e5f4,ffffff,False,7200.0,France,Enabled,fr,Tue May 27 10:38:13 +0000 2008,Paris,985303,22426,435976,1286,business,NaN,4.5657,C0QTG3P4A2DICQ7O.png
1415,2SFQ9KPTBQILL0FA,C0rK,NaN,Set,Verified,0084b4,ddeef6,022330,False,10800.0,"Strasbourg, Alsace",Enabled,fr,Thu Jul 05 13:18:16 +0000 2012,Athens,110733,351,21917,398,celebrity,8.337,1.7701,2SFQ9KPTBQILL0FA.png
4087,5C43961ASN2SH970,C8IEoDGx5ars,https://blob/o6_0ijts:.com,Set,Not verified,ff6600,ffd498,ffffff,False,25200.0,"Jakarta, Indonesia",disabled,en,Mon Jul 16 06:31:30 +0000 2012,Bangkok,2907,182,3132,8,unknown,7.333,0.3590,5C43961ASN2SH970.png


### Functions

In [ ]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))


def clean_profile_category(df):
    ''' replace "  " with "unknown" '''
    return df.apply(lambda x: x.astype(str).str.replace(' ', 'unknown'))


def clean_user_language(df):
    ''' keep only the first two characters '''
    return df.apply(lambda x: x.astype(str).str.split('-').str[0])


def lower_location_visibility(df):
    ''' lowercase location visibility feature '''
    return df.apply(lambda x: x.astype(str).str.lower())


def date2int(df):
    result = pd.to_datetime(df['profile_creation_timestamp']).astype(int)
    return pd.DataFrame(result)


def date2ym(df):
    result = pd.to_datetime(df['profile_creation_timestamp']).dt.year \
             * 100 + pd.to_datetime(df['profile_creation_timestamp']).dt.month
    return pd.DataFrame(result)


def to_string(df):
    return df.apply(lambda x: x.astype(str))


def if_black(df):
    def f(x):
        return x.apply(lambda x: True if x != '000000' else False)
    return df.apply(f, axis=1)


class DenseTransformer(sklearn.base.TransformerMixin):
# https://stackoverflow.com/questions/28384680/scikit-learns-pipeline-a-sparse-matrix-was-passed-but-dense-data-is-required
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [ ]:
score = make_scorer(rmsle, greater_is_better=False)

### Column transformers

#### Groups of features

In [ ]:
numeric_features = [
                    'num_of_followers', 
                    'num_of_people_following',
                    'num_of_status_updates', 
                    'num_of_direct_messages',
                    'avg_daily_profile_visit_duration_in_seconds',
                    'avg_daily_profile_clicks'
                    ]
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=2, weights='distance')),
    ('log', FunctionTransformer(np.log1p)),
    ('scaler', StandardScaler())])

categorical_features = ['utc_offset']
categorical_transformer = Pipeline(steps=[
    ('clean', FunctionTransformer(to_string)),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

boolean_features = ['is_profile_view_size_customized',
                    'profile_cover_image_status',
                    'profile_verification_status']
boolean_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

color_features = ['profile_text_color',
                  'profile_page_color',
                  'profile_theme_color']
color_transformer = Pipeline(steps=[
    ('to_bool', FunctionTransformer(if_black)),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

#### Individual features

In [ ]:
profile_category = ['profile_category']
profile_category_transformer = Pipeline(steps=[
    ('clean', FunctionTransformer(clean_profile_category)),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

user_language = ['user_language']
user_language_transformer = Pipeline(steps=[
    # ('clean', FunctionTransformer(clean_user_language)),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

personal_url = ['personal_url', 'location']
personal_url_transformer = Pipeline(steps=[
    ('not_na', FunctionTransformer(pd.notna)),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

loc_visib = ['location_public_visibility']
loc_visib_transformer = Pipeline(steps=[
    ('lower', FunctionTransformer(lower_location_visibility)),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

time = ['profile_creation_timestamp']
time_transformer = Pipeline(steps=[
    ('extract', FunctionTransformer(date2ym)),
    ('log', FunctionTransformer(np.log1p)),
    ('scaler', MinMaxScaler())])

#### Combine Transformers

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric_features', numeric_transformer, numeric_features),
        ('categorical_features', categorical_transformer, categorical_features),
        ('boolean_features', boolean_transformer, boolean_features),
        ('color_features', color_transformer, color_features),
        # ('profile_category', profile_category_transformer, profile_category),  # drop
        ('user_language', user_language_transformer, user_language),
        ('loc_visib', loc_visib_transformer, loc_visib),
        ('personal_url', personal_url_transformer, personal_url),
        ('time', time_transformer, time)],
        transformer_weights={
            'numeric_features': 1.1,
            'categorical_features': 1.7,
            'boolean_features': 0.9,
            'color_features': 0.3,
            'user_language': 1.1,
            'loc_visib': 0.8,
            'personal_url': 1.85,
            'time': 1.3,
        }
    )

### Target transformer coupled with a regressor

In [ ]:
# regressor = LinearRegression()
# regressor = SVR(C=0.354, epsilon=0.222)
NuSVR_regressor = NuSVR(nu=0.645, C=0.35)
AdaBoost_regressor = AdaBoostRegressor(random_state=0, n_estimators=724,
                              base_estimator=DecisionTreeRegressor(max_depth=17),
                              learning_rate=2.06, loss='exponential')
# regressor = Ridge(fit_intercept=False)
# Lasso_regressor = LassoLars(random_state=1, fit_intercept=False)
# KNeighbors_regressor = KNeighborsRegressor(n_jobs=-1, p=2)
# RandomForest_regressor = RandomForestRegressor(max_features=None)
transformer = QuantileTransformer(n_quantiles=13, output_distribution='normal')

# Trying to improve with the ensemble of all predictors
regressor_vote = VotingRegressor(
    estimators=[
        ('NuSVR', NuSVR_regressor), # NuSVR
        ('AdaBoost', AdaBoost_regressor), # AdaBoost
        # ('Lasso_regressor', Lasso_regressor), # Lasso_regressor
        # ('KNeighbors', KNeighbors_regressor), # KNeighbors
        # ('RandomForest', RandomForest_regressor), # RandomForest
        ],
    # weights=[1, 1, 1, 1, 1]
    )
# clf_vote.fit(X_train, y_train)
# clf_vote.score(X_valid, y_valid)

regr = TransformedTargetRegressor(regressor=regressor_vote,
                                  transformer=transformer)

### Main pipeline

In [ ]:
estimators = [('preprocessor', preprocessor),
              ('to_dense', DenseTransformer()), # Use only for RandomForestRegressor
              # ('dimensions', TruncatedSVD(n_components=50, n_iter=7, random_state=42)),
              # ('dimensions', GenericUnivariateSelect(mode="fpr", param=0.92)),
              ('regressor', regr)]
pipe = Pipeline(estimators)
# pipe.get_params()

### Train on valid and get results

In [ ]:
_ = pipe.fit(X_train, y_train)
predictions = pipe.predict(X_valid)
rmsle_train = rmsle(y_train, pipe.predict(X_train))
rmsle_valid = rmsle(y_valid, predictions)
print(f'Train error: {rmsle_train:.3f}')
print(f'Valid error: {rmsle_valid:.3f}')

Train error: 0.947
Valid error: 1.424


### Grid search for hyperparameters

In [ ]:
imputer = [dict(
    preprocessor__numeric_features__imputer=[SimpleImputer()],
    preprocessor__numeric_features__imputer__strategy=['mean', 'median']),
           dict(
    preprocessor__numeric_features__imputer=[KNNImputer()],
    preprocessor__numeric_features__imputer__n_neighbors=np.arange(1, 11, 1, dtype=int),
    preprocessor__numeric_features__imputer__weights=['uniform', 'distance'])
]
# best: {'preprocessor__numeric_features__imputer': KNNImputer(n_neighbors=3, weights='distance'),
#        'preprocessor__numeric_features__imputer__n_neighbors': 3,
#        'preprocessor__numeric_features__imputer__weights': 'distance'}

In [ ]:
preprocessing = dict(
    preprocessor__numeric_features__scaler=[StandardScaler(), MinMaxScaler(), MaxAbsScaler()],
    preprocessor__time__extract=[FunctionTransformer(date2int), FunctionTransformer(date2ym)],
    preprocessor__user_language__clean=['passthrough', FunctionTransformer(clean_user_language)],
)
# best: {'preprocessor__numeric_features__scaler': StandardScaler(),
#        'preprocessor__time__extract': FunctionTransformer(date2ym),
#        'preprocessor__user_language__clean': 'passthrough'}

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric_features', numeric_transformer, numeric_features),
        ('categorical_features', categorical_transformer, categorical_features),
        ('boolean_features', boolean_transformer, boolean_features),
        ('color_features', color_transformer, color_features),
        # ('profile_category', profile_category_transformer, profile_category),  # drop
        ('user_language', user_language_transformer, user_language),
        ('loc_visib', loc_visib_transformer, loc_visib),
        ('personal_url', personal_url_transformer, personal_url),
        ('time', time_transformer, time)],
        transformer_weights={
            'numeric_features': 1,
            'categorical_features': 0.70,
            'boolean_features': 1,
            'color_features': 1,
            'user_language': 1,
            'loc_visib': 1,
            'personal_url': 1,
            'time': 1,
        }
    )

In [ ]:
f_selection = dict(
    preprocessor__numeric_features=['drop', numeric_transformer],
    preprocessor__categorical_features=['drop', categorical_transformer],
    preprocessor__boolean_features=['drop', boolean_transformer],
    preprocessor__color_features=['drop', color_transformer],
    preprocessor__profile_category=['drop', profile_category_transformer],
    preprocessor__user_language=['drop', user_language_transformer],
    preprocessor__loc_visib=['drop', loc_visib_transformer],
    preprocessor__personal_url=['drop', personal_url_transformer],
    preprocessor__time=['drop', time_transformer],
)
# best: {'preprocessor__categorical_features': 'drop',
#        'preprocessor__profile_category': 'drop'}

In [ ]:
grid_search = GridSearchCV(pipe, param_grid=f_selection, scoring=score, n_jobs=-1)
_ = grid_search.fit(X_train, y_train)
print("Best score:", grid_search.best_score_)
print("Best params:", grid_search.best_params_)

In [ ]:
# df = pd.DataFrame(grid_search.cv_results_)

### Bayesian optimization
Hyperopt's job is to find the best value of a scalar-valued, possibly-stochastic function over a set of possible arguments to that function. Whereas many optimization packages will assume that these inputs are drawn from a vector space, Hyperopt is different in that it encourages you to describe your search space in more detail. By providing more information about where your function is defined, and where you think the best values are, you allow algorithms in hyperopt to search more efficiently.

- [Parameter expressions](https://github.com/hyperopt/hyperopt/wiki/FMin#21-parameter-expressions)
- [Tutorial](https://github.com/WillKoehrsen/hyperparameter-optimization/blob/master/Bayesian%20Hyperparameter%20Optimization%20of%20Gradient%20Boosting%20Machine.ipynb)

In [ ]:
!pip install hyperopt

In [ ]:
import hyperopt
from hyperopt import hp

In [ ]:
svr_space = dict(
    regressor__regressor__kernel=[hp.choice('regressor__regressor__kernel',
                                            ['rbf', 'sigmoid', 'linear', 'poly'])],
    regressor__regressor__coef0=[hp.lognormal('regressor__regressor__coef0', 0, 1)],
    regressor__regressor__degree=[hp.quniform('regressor__regressor__degree', 0, 5, 1)],
    regressor__regressor__C=[hp.lognormal('regressor__regressor__C', 0, 1)],
    regressor__regressor__epsilon=[hp.lognormal('regressor__regressor__epsilon', 0, 1)]
)
# best loss: 1.7437062778333687
# {'regressor__regressor__C': (0.35392641089361354,),
#  'regressor__regressor__degree': (3.0,),
#  'regressor__regressor__kernel': ('rbf',)}

In [ ]:
ada_space = dict(
    regressor__regressor__loss=[hp.choice('regressor__regressor__loss', ['linear', 'square', 'exponential'])],
    regressor__regressor__n_estimators=[hp.quniform('regressor__regressor__n_estimators', 50, 800, 1)],
    regressor__regressor__learning_rate=[hp.lognormal('regressor__regressor__learning_rate', 0, 1)],
    regressor__regressor__base_estimator__max_depth=[hp.quniform('regressor__regressor__base_estimator__max_depth', 2, 20, 1)]
)
# best loss: 1.758543277550394
# {'regressor__regressor__base_estimator__max_depth': (17.0,),
#  'regressor__regressor__learning_rate': (2.060304100430177,),
#  'regressor__regressor__loss': ('exponential',),
#  'regressor__regressor__n_estimators': (724.0,)}

In [ ]:
ridge_space = dict(
    regressor__regressor__alpha=[hp.lognormal('regressor__regressor__alpha', 0, 1)],
    regressor__regressor__solver=[hp.choice('regressor__regressor__solver',
                                            ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])]
)
# best loss: 1.9026832250411743

In [ ]:
lassolars_space = dict(
    regressor__regressor__alpha=[hp.lognormal('regressor__regressor__alpha', 0, 1)],
    # regressor__regressor__eps=[hp.lognormal('regressor__regressor__eps', 0, 1)]
)
# best loss: 1.902702357590335

In [ ]:
nusvr_space = dict(
    regressor__regressor__nu=[hp.uniform('regressor__regressor__nu', 0.0001, 1)],
    # regressor__regressor__kernel=[hp.choice('regressor__regressor__kernel',
    #                                         ['rbf', 'sigmoid', 'linear', 'poly'])],
    # regressor__regressor__coef0=[hp.lognormal('regressor__regressor__coef0', 0, 1)],
    # regressor__regressor__degree=[hp.quniform('regressor__regressor__degree', 0, 5, 1)],
    regressor__regressor__C=[hp.lognormal('regressor__regressor__C', 0, 1)]
)
# best loss: 1.7422090855908432
# {'regressor__regressor__C': (0.35495952136665515,),
#  'regressor__regressor__nu': (0.6818017344641393,)}

In [ ]:
knn_space = dict(
    # regressor__regressor__metric=[hp.choice('regressor__regressor__metric', ['cosine', 'l1', 'l2'])],
    regressor__regressor__n_neighbors=[hp.quniform('regressor__regressor__n_neighbors', 1, 100, 1)],
    # 'regressor__regressor__p': 2,
    regressor__regressor__weights=[hp.choice('regressor__regressor__weights', ['uniform', 'distance'])]
)
# best loss: 1.8704194083689019
# {'regressor__regressor__n_neighbors': (43.0,),
#  'regressor__regressor__weights': ('distance',)}

In [ ]:
rf_space = dict(
    regressor__regressor__n_estimators=[hp.quniform('regressor__regressor__n_estimators', 50, 400, 10)],
    regressor__regressor__criterion=[hp.choice('regressor__regressor__criterion', ['mse', 'mae'])],
    # regressor__regressor__min_samples_split=[]
    regressor__regressor__max_samples=[hp.uniform('regressor__regressor__max_samples', 0, 1)],
    regressor__regressor__min_samples_split=[hp.uniform('regressor__regressor__min_samples_split', 0, 1)],
    regressor__regressor__min_samples_leaf=[hp.uniform('regressor__regressor__min_samples_leaf', 0, 0.5)]
)

In [ ]:
# Rerun after setting pipe and before running the search
def objective(params, X=X_train, y=y_train, pipe=pipe, score=score, cv=5):
    '''
    A "black box" function calculating a score (=the loss to minimize) given
    a set of hyperparameters, using GridSearchCV with cv=5 by default.
    It flattens the hyperparameter spaces (if conditional hyperparams are passed).
    It casts some floats to ints (based on a hardcoded list of hyperparamer names).
    '''
    flat_params = {}
    for k, v in params.items():
        if isinstance(v[0], dict):
            flat_params.update(v[0])
        else:
            flat_params[k] = v
    for param in ['regressor__regressor__degree',
                  'regressor__regressor__n_estimators',
                  'regressor__regressor__n_estimators',
                  'regressor__regressor__n_neighbors',
                  'dimensions__k']:
        if param in flat_params.keys():
            flat_params[param] = [int(flat_params[param][0])]

    grid_search = GridSearchCV(pipe, flat_params, scoring=score, cv=cv, n_jobs=-1)
    _ = grid_search.fit(X, y)
    loss = -grid_search.best_score_
    return {'loss': loss, 'params': flat_params, 'status': hyperopt.STATUS_OK}

In [ ]:
# Pass the appropriate space before running
bayes_trials = hyperopt.Trials()
best = hyperopt.fmin(fn=objective, space=rf_space, algo=hyperopt.tpe.suggest,
                     max_evals=50, trials=bayes_trials)

In [ ]:
# See the best params (pass the appropriate space)
hyperopt.space_eval(rf_space, best)

In [ ]:
# hyperopt.pyll.stochastic.sample(ada_space)

### Train on whole data

In [ ]:
X_train_whole = pd.concat([X_train, X_valid])
y_train_whole = pd.concat([y_train, y_valid])
pipe.fit(X_train_whole, y_train_whole)
pred_submission = pipe.predict(test_data)

### Save Predictions

In [ ]:
output = pd.DataFrame({'Id': test_data['id'], 
                       'Predicted': np.floor(pred_submission).astype('int')})
output.to_csv('submission.csv', index=False)